## Architecture
GOAL: modify transportation graph to result in more connected social networks

Graph G -> node encoder (trainable) -> 
        classify network by social network strength /
        propose attribute changes for edges & nodes to minimize social loss criterion
            (loss calculated via simulation?)


- does the node encoder have to be retrained for each new graph?

GraphSAGE: predicts embedding of new node without retraining (dynamic graph)

    Propose new nodes with embedding similarity with preexisting nodes that are social hubs


- graph embedding
    - gmel?
- action is passing modified edge / node attr (make additional attr?)
    - make into cleaner vector to be passed as state
- generate k candidate modification -> compute reward -> choose best -> next state

- env:
    - edges: id, general, length (speed and capacity implicit?)
        - precalc betweenness?
    - input embedding (edges):
        - [ general, length, betweenness, 0, ..., 0 ]
- action:
    - modified node general or edge general